In [52]:
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

# 不同的Memory工具
- 短时记忆：存储在内存中
- 构建记忆实体清单
- 接入知识图谱
- 长对话在内存中的处理方式
- 长时记忆实现方式

In [53]:
from langchain.memory import  ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("你好，我是人类！")
memory.chat_memory.add_ai_message("你好，我是AI，有什么可以帮助你的吗？")

memory.load_memory_variables({})

{'history': 'Human: 你好，我是人类！\nAI: 你好，我是AI，有什么可以帮助你的吗？'}

# 实现一个最近的对话窗口，超过窗口条数的对话将被删除



In [54]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(memory_key="chat_history", k=3)
memory.save_context({"input":"你好，我是人类!"},{"output":"你好，我是AI，有什么可以帮助你的吗？"})
memory.save_context({"input":"我想吃鸡肉"},{"output":"好的，我帮你找找鸡肉的做法"})

memory.load_memory_variables({})

{'chat_history': 'Human: 你好，我是人类!\nAI: 你好，我是AI，有什么可以帮助你的吗？\nHuman: 我想吃鸡肉\nAI: 好的，我帮你找找鸡肉的做法'}

In [55]:
# 构建记忆实体概念清单

from langchain.chat_models import  ChatOpenAI
from langchain.memory import ConversationEntityMemory

llm = ChatOpenAI(
    temperature=0
)

memory = ConversationEntityMemory(llm=llm)
_input = {
    "input":"胡八一和王胖子雪莉杨经常在一起冒险，合称盗墓铁三角."
}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {
        "output":"听起来很刺激，我也想加入他们！"
    }
)



memory.entity_cache


KeyboardInterrupt: 

In [ ]:
memory.load_memory_variables({"input":"盗墓铁三角是谁？"})

{'history': 'Human: 胡八一和王胖子雪莉杨经常在一起冒险，合称盗墓铁三角.\nAI: 听起来很刺激，我也想加入他们！',
 'entities': {'盗墓铁三角': '盗墓铁三角是由胡八一、王胖子和雪莉杨组成的冒险团队。'}}

In [ ]:
memory.load_memory_variables({"input":"雪莉杨是谁？"})

{'history': 'Human: 胡八一和王胖子雪莉杨经常在一起冒险，合称盗墓铁三角.\nAI: 听起来很刺激，我也想加入他们！',
 'entities': {'雪莉杨': '雪莉杨是与胡八一和王胖子一起冒险的一部分，合称盗墓铁三角。'}}

# 使用知识图谱构建记忆

In [ ]:
from langchain.chat_models import  ChatOpenAI
from langchain.memory import ConversationKGMemory

llm = ChatOpenAI(
    temperature=0
)   

memory = ConversationKGMemory(llm=llm,return_messages=True)

memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)

memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)

In [ ]:
memory.load_memory_variables({"input":"tomie是谁?"})

{'history': [SystemMessage(content='On tomie: tomie 是一个 培训讲师.')]}

In [ ]:
memory.get_current_entities("tomie最喜欢做什么事?")

['tomie']

In [ ]:
memory.get_knowledge_triplets("tomie最喜欢打游戏")

[KnowledgeTriple(subject='tomie', predicate='最喜欢', object_='打游戏')]

# 长对话在内存中的处理方式：总结摘要以及token计算

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import  ChatOpenAI

llm = ChatOpenAI(
    temperature=0
)

memory = ConversationSummaryMemory(llm=llm)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)

In [ ]:
memory.load_memory_variables({})

{'history': 'The human asks the AI to help find Tomie, who is a training instructor. The AI apologizes and asks for more information about Tomie.'}

In [ ]:
messages = memory.chat_memory.messages
#print(messages)
memory.predict_new_summary(messages,"")

'The human asks the AI to help find Tomie. The AI asks for clarification on who or what Tomie is. The human explains that Tomie is a training instructor. The AI acknowledges and understands.'

In [ ]:
#使用ChatMessageHistory来快速获得对话摘要
from langchain.memory import ConversationSummaryMemory
from langchain.memory import ChatMessageHistory
from langchain.chat_models import  ChatOpenAI

hisiory = ChatMessageHistory()
hisiory.add_user_message("你好，我是人类！")
hisiory.add_ai_message("你好，我是AI小丸子，有什么可以帮助你的吗？")

# memory = ConversationSummaryMemory.from_messages(
#    llm=ChatOpenAI(temperature=0),
#    chat_memory=hisiory,
#    return_messages=True
# )

memory = ConversationSummaryMemory(
    llm=ChatOpenAI(temperature=0),
    return_messages=True,
    buffer="\nThe AI introduces itself as AI Little Maruko and asks if there is anything it can help the human with.",
    chat_memory=hisiory
)

In [ ]:
memory.buffer
memory.load_memory_variables({})

{'history': [SystemMessage(content='\nThe AI introduces itself as AI Little Maruko and asks if there is anything it can help the human with.')]}

- 当对话持续进行且对话内容很多的时候
- 可以使用ConversationSummaryBufferMemory来存储对话摘要
- 这是一种非常有用的方式,它会根据token的数量来自动判断是否需要进行摘要
- 当token数量超过阈值的时候,会自动进行摘要
- 在缓冲区中,会保留最近的k条对话
- 比较久的对话会被删除，在删除前会进行摘要

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import  ChatOpenAI

llm = ChatOpenAI(temperature=0)


#max_token_limit的作用是设置进行memory总结的阈，单纯比较Token数量是没有意义的，
#这里更多的是避免memory占用了过多的上下文窗口token，且要保证对话的连贯（要把对话背景可以完整的告诉LLM），
#另外在实际应用里，这种都会做提示词约束，比如总结为多少字以内的对话。
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=100,
    return_messages=True
)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)

In [ ]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human asks the AI to help find Tomie.'),
  AIMessage(content='对不起请问什么是tomie？'),
  HumanMessage(content='tomie是一个培训讲师'),
  AIMessage(content='好的，我知道了。'),
  HumanMessage(content='今天他要讲一门关于RAG的课程'),
  AIMessage(content='好的，我知道了。需要RAG的资料吗？')]}

# Conversation Token Buffer使用token长度来决定什么时候刷新内存

In [ ]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)
memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=100
)
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)
memory.save_context(
    {"input":"不需要资料了，谢谢"},
    {"output":"好的，那我就不打扰你了。"}
)

In [ ]:
memory.load_memory_variables({})

{'history': 'AI: 好的，我知道了。\nHuman: 今天他要讲一门关于RAG的课程\nAI: 好的，我知道了。需要RAG的资料吗？\nHuman: 不需要资料了，谢谢\nAI: 好的，那我就不打扰你了。'}

# 长时记忆的实现方式
- 通过向量数据库来存储之前的对话内容，有的向量数据库服务还提供自动摘要等，每次对话的时候，都会从向量数据库里查询最相关的文档或历史对话


In [56]:
from langchain_openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS

In [57]:
memory = ConversationBufferMemory()
memory.save_context(
    {"input":"帮我找一下tomie"},
    {"output":"对不起请问什么是tomie？"}
)
memory.save_context(
    {"input":"tomie是一个培训讲师"},
    {"output":"好的，我知道了。"}
)
memory.save_context(
    {"input":"今天他要讲一门关于RAG的课程"},
    {"output":"好的，我知道了。需要RAG的资料吗？"}
)
memory.save_context(
    {"input":"不需要资料了，谢谢"},
    {"output":"好的，那我就不打扰你了。"}
)

vectorstore = FAISS.from_texts(
    memory.buffer.split("\n"),
    OpenAIEmbeddings()
)
FAISS.save_local(vectorstore,"test_faiss")

In [59]:
FAISS.load_local(
    "test_faiss",OpenAIEmbeddings(model="text-embedding-3-small"),
    allow_dangerous_deserialization=True
).similarity_search("tomie是什么职业?")

[Document(page_content='Human: 今天他要讲一门关于RAG的课程'),
 Document(page_content='AI: 好的，我知道了。需要RAG的资料吗？'),
 Document(page_content='Human: tomie是一个培训讲师'),
 Document(page_content='AI: 好的，那我就不打扰你了。')]

In [62]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.memory import VectorStoreRetrieverMemory
r1 = FAISS.load_local("test_faiss",OpenAIEmbeddings(),allow_dangerous_deserialization=True)
r2 = r1.as_retriever(
    search_kwargs={"k":1}
)
memory2 = VectorStoreRetrieverMemory(
    retriever=r2
)
memory2.load_memory_variables({"prompt":"tomie是什么职业?"})

{'history': 'Human: tomie是一个培训讲师'}